# Exercise Chapter 2 
Industrial Statistics: A Computer Based Approach with Python<br>
by Ron Kenett, Shelemyahu Zacks, Peter Gedeck

Publisher: Springer International Publishing; 1st edition (2023) <br>
<!-- ISBN-13: 978-3031075650 -->

(c) 2022 Ron Kenett, Shelemyahu Zacks, Peter Gedeck

The code needs to be executed in sequence.

In [ ]:
import os
os.environ['OUTDATED_IGNORE'] = '1'
import warnings
from outdated import OutdatedPackageWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=OutdatedPackageWarning)

In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as sms
from statsmodels.graphics.mosaicplot import mosaic
import seaborn as sns
import matplotlib.pyplot as plt
import pwlf

import mistat

# Exercise 1

In [ ]:
oelect = mistat.load_data('OELECT')
qcc = mistat.QualityControlChart(oelect, qcc_type='xbarone',
                                 std_dev='SD')
qcc.plot()
plt.show()

# Exercise 2

In [ ]:
steelrod = mistat.load_data('STEELROD')
qcc = mistat.QualityControlChart(steelrod, qcc_type='xbarone',
                                 std_dev='SD')
qcc.plot()
plt.show()

# Exercise 4

In [ ]:
oturb2 = mistat.load_data('OTURB2')
# print(oturb2)
sd = np.sqrt(oturb2['xbar'].var() / 5)
center = oturb2['xbar'].mean()
print(sd, center)
qcc = mistat.QualityControlChart(oturb2['xbar'], qcc_type='xbarone',
        center=center, std_dev=sd)
qcc.plot()
plt.show()

# Exercise 6

In [ ]:
oelect = mistat.load_data('OELECT')

qcc = mistat.QualityControlChart(oelect, qcc_type='xbarone',
                                 std_dev='SD')
pc = mistat.ProcessCapability(qcc, spec_limits = [210, 230])
pc.plot()
plt.show()
pc.summary()

# Exercise 7

In [ ]:
oelect = mistat.load_data('OELECT')
qcc = mistat.QualityControlChart(oelect, qcc_type='xbarone',
    std_dev=4.004, center=219.25)
pc = mistat.ProcessCapability(qcc, spec_limits = [210, 230],
                              confidence_level=0.975)
pc.summary()
pc = mistat.ProcessCapability(qcc, spec_limits = [210, 230],
                              confidence_level=0.95)
pc.summary()

In [ ]:
def confidenceLimitSL(Cp, n, alpha):
    F = stats.f(1, n-1).ppf(1-(1-alpha)/2)
    a = np.sqrt(F/n) * np.sqrt(Cp**2/2 + (1 - F/(2*n))/9)
    b = 1 - F/(2*n)
    return (Cp - a) / b, (Cp + a )/ b

n = len(oelect)
rho_1L, rho_1U = confidenceLimitSL(pc.Cp_l, n, 0.95)
rho_2L, rho_2U = confidenceLimitSL(pc.Cp_u, n, 0.95)
print(rho_1L, rho_1U)
print(rho_2L, rho_2U)

# Exercise 8

In [ ]:
steelrod = mistat.load_data('STEELROD')
qcc = mistat.QualityControlChart(steelrod, qcc_type='xbarone',
                                 std_dev='SD')
pc = mistat.ProcessCapability(qcc, spec_limits = [19, 21],
                              confidence_level=0.95)
pc.summary()

In [ ]:
n = len(steelrod)
rho_1L, rho_1U = confidenceLimitSL(pc.Cp_l, n, 0.95)
rho_2L, rho_2U = confidenceLimitSL(pc.Cp_u, n, 0.95)
print(rho_1L, pc.Cp_l, rho_1U)
print(rho_2L, pc.Cp_u, rho_2U)

# Exercise 9

In [ ]:
settings = {'m': 30, 's': 0.005, 'k': 1000, 't': 290, 
            'p0': 90_000, 'v0': 0.002, 't0': 340}

simulator = mistat.PistonSimulator(n_simulation=20, n_replicate=1, seed=1, 
                                   **settings)
Ps = simulator.simulate()
cycleTime = mistat.qcc_groups(Ps['seconds'], Ps['group'])

qcc = mistat.QualityControlChart(cycleTime, std_dev= Ps['seconds'].std())
print(f'Mean    {qcc.center:.4f}')
print(f'Std.Dev {qcc.std_dev:.4f}')

In [ ]:
pc = mistat.ProcessCapability(qcc, spec_limits = [0.04, 0.06],
                              confidence_level=0.95)
print(f'C_p {pc.Cp:.3f}')

In [ ]:
settings = {'m': 60, 's': 0.02, 'k': 5_000, 't': 296, 
            'p0': 110_000, 'v0': 0.01, 't0': 360}

simulator = mistat.PistonSimulator(n_simulation=20, n_replicate=1, seed=1, **settings)
Ps = simulator.simulate()
cycleTime = mistat.qcc_groups(Ps['seconds'], Ps['group'])
qcc = mistat.QualityControlChart(cycleTime, std_dev= Ps['seconds'].std())
print(f'Mean    {qcc.center:.4f}')
print(f'Std.Dev {qcc.std_dev:.4f}')

In [ ]:
pc = mistat.ProcessCapability(qcc, spec_limits = [0.04, 0.06],
                              confidence_level=0.95)
pc.summary()

# Exercise 12

In [ ]:
october = pd.DataFrame([
  ['Missing component', 293],
  ['Wrong component', 431],
  ['Too much solder', 120],
  ['Insufficient solder', 132],
  ['Failed component', 183],
], columns=['Issue', 'Count'])
november = pd.DataFrame([
  ['Missing component', 34],
  ['Wrong component', 52],
  ['Too much solder', 25],
  ['Insufficient solder', 34],
  ['Failed component', 18],
], columns=['Issue', 'Count'])

def makeParetoChart(data, ax, title):
  paretoChart = mistat.ParetoChart(data['Count'], labels=data['Issue'])
  paretoChart.plot(rotation=30, ha='right', ax=ax)
  ax.set_title(title)

fig, axes = plt.subplots(ncols=2, figsize=(8,4))
makeParetoChart(october, axes[0], 'October')
makeParetoChart(november, axes[1], 'November (2nd week)')
fig.suptitle('')
plt.tight_layout()
plt.show()

# Exercise 14

In [ ]:
settings = {'m': 30, 's': 0.005, 'v0': 0.002, 'k': 1000,
            'p0': 90_000, 't': 290, 't0': 340}

simulator = mistat.PistonSimulator(n_simulation=20, n_replicate=5, seed=1, **settings)
Ps = simulator.simulate()

# Add 0.02 seconds to last 50 simulation results
Ps.loc[50:,'seconds'] = Ps.loc[50:,'seconds'] + 0.02

def makeQCCplot(data, reference, qcc_type, title):
  # convert to groups
  data = mistat.qcc_groups(data['seconds'], data['group'])
  reference = mistat.qcc_groups(reference['seconds'], reference['group'])
  # calculate control limits based on reference data
  qcc_ref = mistat.QualityControlChart(reference, qcc_type=qcc_type)
  qcc = mistat.QualityControlChart(data, qcc_type=qcc_type,
    center=qcc_ref.center, limits=qcc_ref.limits)
  return qcc.plot(title=title) 

# create xbar and R control charts
reference = Ps.iloc[:50, ]

makeQCCplot(Ps, reference, 'xbar', 'for cycleTime')
plt.show()
makeQCCplot(Ps, reference, 'R', 'for cycleTime')
plt.show()

In [ ]:
# Create random numbers from U(0,1)
R = stats.uniform.rvs(size=100)

# sort cycle times using by the order of the random numbers 
# this randomizes the cycle times 
Ps['seconds'] = Ps['seconds'][R.argsort()].values
# alternative versions
# Ps['seconds'] = Ps['seconds'].sample(frac=1).values
# Ps['seconds'] = [s for _, s in sorted(zip(R, Ps['seconds']))]

makeQCCplot(Ps, Ps, 'xbar', 'for cycleTime')
plt.show()
makeQCCplot(Ps, Ps, 'R', 'for cycleTime')
plt.show()

# Exercise 15

In [ ]:
settings = {'m': 30, 's': 0.005, 'v0': 0.002, 'k': 1000,
            'p0': 90_000, 't': 290, 't0': 340}

simulator = mistat.PistonSimulator(n_simulation=20, n_replicate=5, seed=1, **settings)
Ps = simulator.simulate()
data = mistat.qcc_groups(Ps['seconds'], Ps['group'])
for qcc_type in ('xbar', 'S'):
  qcc = mistat.QualityControlChart(data, qcc_type=qcc_type)
  print(f'{qcc_type:4s} Center {qcc.center:.4f} Control limits ' +
        f'[{qcc.limits.LCL[0]:.5f}, {qcc.limits.UCL[0]:.4f}]')

In [ ]:
simulator = mistat.PistonSimulator(n_simulation=20, n_replicate=10, seed=1, **settings)
Ps = simulator.simulate()
data = mistat.qcc_groups(Ps['seconds'], Ps['group'])
for qcc_type in ('xbar', 'S'):
  qcc = mistat.QualityControlChart(data, qcc_type=qcc_type)
  print(f'{qcc_type:4s} Center {qcc.center:.4f} Control limits ' +
        f'[{qcc.limits.LCL[0]:.5f}, {qcc.limits.UCL[0]:.4f}]')

# Exercise 16

In [ ]:
data = mistat.load_data('PROCESS_SEGMENT')

def sensorData(data, label):
    series = data[label]
    return pd.DataFrame({
        'Time': np.arange(len(series)),
        'values': series,
    })

sensorX = sensorData(data, 'X')
sensorZ = sensorData(data, 'Z')

def fitPiecewiseLinearFit(sensor, knots):
    model = pwlf.PiecewiseLinFit(sensor['Time'], sensor['values'], degree=1)
    model.fit(knots)
    return model

modelX = fitPiecewiseLinearFit(sensorX, 6)
modelZ = fitPiecewiseLinearFit(sensorZ, 3)

def plotPiecewiseLinearFit(sensor, model, ax, label):
    for bp in model.fit_breaks[1:-1]:
        ax.axvline(bp, color='lightgrey')
    ax.scatter(sensor['Time'], sensor['values'], color='grey', alpha=0.5)
    ax.plot(sensor['Time'], model.predict(sensor['Time']), color='black')
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    return ax
fig, axes = plt.subplots(ncols=2, figsize=(8,4))
plotPiecewiseLinearFit(sensorX, modelX, axes[0], 'Sensor X')
plotPiecewiseLinearFit(sensorZ, modelZ, axes[1], 'Sensor Z')
plt.tight_layout()
plt.show()

In [ ]:
def getResultTable(model):
    predictions = model.predict(model.fit_breaks).round(1)
    df = pd.DataFrame({
        'breaks': model.fit_breaks[:-1], 
        'slope': model.calc_slopes(),
        'prediction': [f'{p1}--{p2}' for p1, p2 in zip(predictions[:-1], predictions[1:])]
    })
    borders = list(df['breaks'].round(1))
    left = borders[1:]
    right = borders[2:]
    Range = [f'${l} \\leq\mathrm{{Time}}< {r}$' for l, r in zip(left, right)]
    Range.insert(0, f'$\\mathrm{{Time}}< {right[0]}$')
    Range.append(f'${left[-1]} \\leq \\mathrm{{Time}}$')
    df.insert(0, 'Range', Range)
    df.index = [f'Segment {idx + 1}' for idx in df.index]
    df = df.drop(columns='breaks')
    return df
df = getResultTable(modelX)

In [ ]:
df['Range'] = [s.replace('$', '').replace('\leq', '<=').replace('\mathrm{Time}', ' Time ') for s in df['Range']]
df

In [ ]:
df = getResultTable(modelZ)

In [ ]:
df['Range'] = [s.replace('$', '').replace('\leq', '<=').replace('\mathrm{Time}', ' Time ') for s in df['Range']]
df